<a href="https://colab.research.google.com/github/ShuqairABD/UPDATED_Gradio_for_ResNet152_Dicom_jpg_png/blob/main/Abdalla_Shuqair/%D0%9E%D0%B1%D0%BD%D0%BE%D0%B2%D0%BB%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9_Gradio_%D0%B4%D0%BB%D1%8F_ResNet_152_Dicom_jpg_png.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **start**

In [ ]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 865.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.8 MB/s eta 0:00:00


In [ ]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.3 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import torch
from torchvision import models, transforms
import torch.nn as nn
import os
import numpy as np
import pydicom
from PIL import Image

import os
import zipfile
import tempfile

import torchvision.transforms as transforms
from torchvision import models


## **Gradio Dicom**

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ResNet-152 with weights! (можно с весами)!!
model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V2)
#model = models.resnet152(weights=None)

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 1),
    nn.Sigmoid()
).to(device)

model_path = '/content/drive/MyDrive/bouns_fine_tuned_resnet152.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

label_mapping = {0: 'Healthy', 1: 'UnHealthy'}

def classify_zip(zip_file):
    results = []
    with tempfile.TemporaryDirectory() as tmpdirname:
        # ZIP
        with zipfile.ZipFile(zip_file.name, 'r') as zip_ref:
            zip_ref.extractall(tmpdirname)

        for root, _, files in os.walk(tmpdirname):
            for filename in files:
                if filename.lower().endswith('.dcm'):
                    dicom_image = pydicom.dcmread(os.path.join(root, filename))
                    image_array = dicom_image.pixel_array

                    if image_array.max() > 1:
                        image_array = (image_array / image_array.max() * 255).astype(np.uint8)
                    image = Image.fromarray(image_array).convert('RGB')

                    input_image = transform(image).unsqueeze(0).to(device)


                    with torch.no_grad():
                        output = model(input_image)
                    predicted = (output > 0.5).float()
                    predicted_label = predicted.item()
                    predicted_class_name = label_mapping[int(predicted_label)]

                    # результат ------>> list
                    results.append((image, f"{filename}: {predicted_class_name}"))


    return results

# Gradio
iface = gr.Interface(
    fn=classify_zip,
    inputs=gr.components.File(label="Загрузить zip-файл DICOM"),
    outputs=gr.components.Gallery(label="Предсказанные классы"),
    title="Классификация патология / норма с помощью ResNet-152",
    description="Загрузить ZIP-файл, содержащий файлы DICOM, чтобы классифицировать их как норма или патология"
)

iface.launch()


## **Gradio Dicom/png/jpg**

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Resnet-152
model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V2)
#model = models.resnet152(weights=None)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 1),
    nn.Sigmoid()
).to(device)

model_path = '/content/drive/MyDrive/bouns_fine_tuned_resnet152.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

label_mapping = {0: 'Healthy', 1: 'UnHealthy'}

def classify_zip(zip_file):
    results = []
    try:
        with tempfile.TemporaryDirectory() as tmpdirname:

            with zipfile.ZipFile(zip_file.name, 'r') as zip_ref:
                zip_ref.extractall(tmpdirname)

            for root, _, files in os.walk(tmpdirname):
                for filename in files:
                    image_path = os.path.join(root, filename)

                    try:
                        # zip Dicom
                        if filename.lower().endswith('.dcm'):
                            dicom_image = pydicom.dcmread(image_path)
                            image_array = dicom_image.pixel_array

                            if image_array.max() > 1:
                                image_array = (image_array / image_array.max() * 255).astype(np.uint8)
                            image = Image.fromarray(image_array).convert('RGB')

                        # zip jpg/png
                        elif filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                            image = Image.open(image_path).convert('RGB')

                        else:
                            continue

                        input_image = transform(image).unsqueeze(0).to(device)
                        with torch.no_grad():
                            output = model(input_image)

                        predicted = (output > 0.5).float()
                        predicted_label = predicted.item()
                        predicted_class_name = label_mapping[int(predicted_label)]

                        results.append((image, f"{filename}: {predicted_class_name}"))

                    except Exception as e:
                        results.append((None, f"Ошибка при обработке {filename}: {str(e)}"))

    except Exception as e:
        return [(None, f"Ошибка при распаковке ZIP-файла: {str(e)}")]

    return results

# Gradio
iface = gr.Interface(
    fn=classify_zip,
    inputs=gr.File(label="Загрузить zip-файл DICOM, png, jpg"),
    outputs=gr.Gallery(label="Предсказанные классы"),
    title="Классификация патология / норма с помощью ResNet-152",
    description="Загрузите ZIP-файл, содержащий файлы DICOM, jpg, png.. чтобы классифицировать их как норму или патологию"
)

iface.launch()
